# Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [3]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [6]:
#!pip install pandas==2.2.2

In [7]:
import pandas as pd
df = pd.read_csv('Data.csv')

In [8]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain_groq import ChatGroq

In [32]:
#llm = ChatOpenAI(temperature=0.9, model=llm_model)


llm = ChatGroq(
    model= "llama-3.2-11b-vision-preview",
    #model = "llama3-groq-70b-8192-tool-use-preview",
    #model = "llama-3.2-90b-vision-preview",
    #model="gemma2-9b-it",
    #model="gemma-7b-it",
    #model="llama3-groq-8b-8192-tool-use-preview",
    #model="distil-whisper-large-v3-en", ## huggingface 
    #model="llama-3.1-70b-versatile",
    #model="llava-v1.5-7b-4096-preview",
    #model="llama3-70b-8192",
    #model="whisper-large-v3",
    temperature=0
)

In [13]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [14]:
chain = LLMChain(llm=llm, prompt=prompt)

C:\Users\jefer\AppData\Local\Temp\ipykernel_14432\1305865249.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [16]:
product = "Queen Size Sheet Set"
chain.run(product)

"Here are some suggestions for a company name that makes Queen Size Sheet Set:\n\n1. **Regal Bedding Co.**: This name conveys a sense of luxury and grandeur, fitting for a Queen Size sheet set.\n2. **Queen's Comfort**: This name emphasizes the comfort and quality of the product, while also referencing the Queen Size.\n3. **Royal Slumber**: This name evokes a sense of relaxation and tranquility, perfect for a company that makes bedding.\n4. **Dreamweaver Bedding**: This name suggests that the company's products will help customers create their own dreams and sleep peacefully.\n5. **Luxury Linens**: This name emphasizes the high-quality materials and craftsmanship that go into the company's products.\n6. **Sleep Haven**: This name creates a sense of a safe and cozy haven, perfect for a company that makes bedding.\n7. **Queen's Quilt Co.**: This name references the Queen Size and suggests a focus on quilting and craftsmanship.\n8. **Slumber & Co.**: This name is simple and straightforward

## SimpleSequentialChain

In [17]:
from langchain.chains import SimpleSequentialChain

In [21]:
#llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [22]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [23]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [24]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Here are some suggestions for a company name that makes Queen Size Sheet Set:

1. **Regal Bedding Co.**: This name suggests luxury and grandeur, fitting for a Queen Size sheet set.
2. **Queen's Comfort**: This name emphasizes the comfort and comfortability of the product.
3. **Royal Slumber**: This name evokes a sense of luxury and relaxation, perfect for a company that makes high-quality bedding.
4. **DreamWeaver Bedding**: This name suggests that the company helps customers create their own dreamy sleep experiences.
5. **Luxury Linens**: This name emphasizes the high-quality materials and craftsmanship of the product.
6. **Queenly Sheets**: This name is straightforward and emphasizes the product's size and quality.
7. **Sleep Royale**: This name suggests that the company offers a regal and luxurious sleep experience.
8. **Comfort Kingdom**: This name emphasizes the comfort and comfortability of the product.
9. **Bedding Bliss**: This na

"Here's a 20-word description for each company:\n\n1. **Regal Bedding Co.**: Luxury bedding brand offering high-quality Queen Size sheet sets for a regal sleeping experience and grand comfort.\n2. **Queen's Comfort**: Providing plush Queen Size sheet sets for ultimate comfort and relaxation, ensuring a restful night's sleep every time.\n3. **Royal Slumber**: Luxury bedding experts crafting high-quality Queen Size sheet sets for a regal and serene sleeping experience, every night.\n4. **DreamWeaver Bedding**: Weaving dreams into reality with luxurious Queen Size sheet sets, designed to create a restful and rejuvenating sleep experience.\n5. **Luxury Linens**: Offering premium Queen Size sheet sets made from high-quality materials, ensuring a luxurious and comfortable sleeping experience.\n6. **Queenly Sheets**: Providing high-quality Queen Size sheet sets for a comfortable and restful night's sleep, fit for royalty.\n7. **Sleep Royale**: Experience the luxury of a regal sleep with our h

## SequentialChain

In [25]:
from langchain.chains import SequentialChain

In [33]:
#llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )


In [34]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )


In [35]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )


In [36]:

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [37]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [38]:
review = df.Review[5]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': 'Voici la traduction de la critique en anglais :\n\n"I trouve le goût médiocre. La mousse ne tient pas, c\'est bizarre. J\'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?"\n\nTraduction :\n\n"I find the taste mediocre. The mousse doesn\'t hold, it\'s strange. I buy the same ones in stores and the taste is much better...\nOld batch or counterfeit?"\n\nNote : La critique semble être négative et suggère que le produit est de mauvaise qualité ou peut être une contrefaçon.',
 'summary': "La critique est négative, car le produit a un goût médiocre, la mousse ne tient pas et le goût est meilleur lorsqu'il est acheté en magasin, ce qui suggère une mauvaise qualité ou une contrefaçon.",
 'followup_message': 'Voici une réponse de suivi :\n\n"Je suis désolé

## Router Chain

In [39]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [40]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [41]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [43]:
#llm = ChatOpenAI(temperature=0, model=llm_model)

llm = ChatGroq(
    model="llama-3.2-11b-vision-preview",
    temperature=0
)

In [44]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [45]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [46]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [52]:
destinations_str

'physics: Good for answering questions about physics\nmath: Good for answering math questions\nHistory: Good for answering history questions\ncomputer science: Good for answering computer science questions'

In [47]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [48]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

C:\Users\jefer\AppData\Local\Temp\ipykernel_14432\3038952769.py:1: LangChainDeprecationWarning: Use RunnableLambda to select from multiple prompt templates. See example in API reference: https://api.python.langchain.com/en/latest/chains/langchain.chains.router.multi_prompt.MultiPromptChain.html
  chain = MultiPromptChain(router_chain=router_chain,


In [49]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'Black body radiation: definition and explanation'}
> Finished chain.


"Black body radiation is a fundamental concept in physics, particularly in thermodynamics and quantum mechanics. \n\nA black body is an idealized object that absorbs all electromagnetic radiation that falls on it, without reflecting or transmitting any of it. In other words, it's a perfect absorber of radiation. This means that when light hits a black body, it absorbs all the energy, and the object heats up.\n\nBlack body radiation refers to the thermal radiation emitted by a black body at a given temperature. This radiation is a result of the thermal motion of the particles in the object, which causes them to emit photons. The energy of these photons is related to the temperature of the object, and the radiation is characterized by a specific spectrum, known as the black body spectrum.\n\nThe black body spectrum is a plot of the intensity of the radiation versus the wavelength of the radiation. It's a bell-shaped curve, with the peak intensity occurring at a specific wavelength, known

In [50]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


"A simple yet great question to start with.\n\nTo answer this question, I'll break it down into its component parts:\n\n1. What is 2 + 0? (This is the first part of the equation)\n   Answer: 2\n\n2. What is 2 + 1? (This is the second part of the equation)\n   Answer: 3\n\n3. What is 2 + 2? (This is the final part of the equation)\n   To find the answer, I'll add the answers from the previous two parts:\n   2 + 3 = 5\n\nSo, the final answer is: 5"

In [51]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is the role of DNA in cells?'}
> Finished chain.


'DNA, or deoxyribonucleic acid, is the fundamental molecule that contains the genetic instructions for the development, growth, and function of all living organisms. It\'s often referred to as the "blueprint" or "instructions" for life.\n\nIn cells, DNA serves several key roles:\n\n1. **Genetic information storage**: DNA contains the genetic code that determines the characteristics of an organism, such as eye color, hair color, height, and other traits.\n2. **Cellular reproduction**: During cell division, DNA is replicated and passed on to the new cells, ensuring that the genetic information is preserved and transmitted to the next generation.\n3. **Gene expression**: DNA is transcribed into RNA (ribonucleic acid), which is then translated into proteins, the building blocks of cells and tissues.\n4. **Cellular regulation**: DNA contains regulatory elements that control gene expression, allowing cells to respond to environmental changes and adapt to different conditions.\n\nIn summary, 

Reminder: Download your notebook to you local computer to save your work.